<p>
<font size='5' face='Georgia, Arial'>IIC-2233 Apunte Programación Avanzada</font><br>
<font size='1'> Equipo Docente IIC2233 2025</font>
</p>

# Tabla de contenidos

1. [Ejecución de código de ejemplo del material que usa PyQt](#Ejecución-de-código-de-ejemplo-del-material-que-usa-PyQt)
2. [*Threads* y PyQt](#Threads-y-PyQt)
3. [*QThread*](#QThread)
    1. [*QThread* con su propia señal](#QThread-con-su-propia-señal)
    2. [Precaución con "donde" se conecta una señal](#Precaución-con-"donde"-se-conecta-una-señal)
    3. [Múltiples *QThread*](#Múltiples-QThread)
    4. [*QMutex*](#QMutex)
5. [Ejemplo Aplicado](#Ejemplo-Aplicado)

## Ejecución de código de ejemplo del material que usa PyQt

Lamentablemente, Jupyter no fue creado con la intención de ejecutar código de GUI de escritorio, por lo que se hace difícil ejecutar e interactuar con las interfaces a través de esta herramienta. Para entender los ejemplos de código relacionados con PyQt, **recomendamos fuertemente** que ejecutes los *scripts* de interfaces gráficas desde tu propio computador, y **NO** en este *notebook*. Para esto, se agregó una carpeta llamada `scripts/` que tiene cada código que se mostrará en este y los siguientes _notebooks_ de esta semana.

**Importante:** Al momento de ejecutar el código, asegúrate de que tu terminal esté posicionada en el mismo directorio donde está el archivo `.py` a ejecutar.

## *Threads* y PyQt

Esta semana pasaremos de una programación secuencial con un único hilo (*thread*) de ejecución con solo interacción por consola y archivos, a un modelo *multithreaded* orientado a interactuar con el usuario mediante distintos tipos de *inputs* gráficos.

El uso de *theading* en conjunto a PyQt es absolutamente compatible, y, si queremos hacer programas de mayor complejidad, a veces es necesario.

Al igual que para programas con un solo *thread*, podemos hacer uso de señales personalizadas en una aplicación *multithreaded*. El siguiente ejemplo muestra cómo crear una señal que controla la acción del *thread* sobre el formulario mediante el método `actualizar_labels()`. El *thread* por su parte, recibe como parámetro la señal y emite mensajes.

Este código se encuentra en el archivo `1-pyqt-qthreads/1_thread.py`

```python
import sys
from threading import Thread
from time import sleep
from PyQt5.QtCore import pyqtSignal
from PyQt5.QtWidgets import QApplication, QWidget, QLabel, QPushButton


class MiThread(Thread):
    """
    Esta clase representa un thread personalizado que será utilizado durante
    la ejecución de la GUI.
    """

    def __init__(self, senal_actualizar: pyqtSignal) -> None:
        super().__init__()
        self.senal_actualizar = senal_actualizar

    def run(self) -> None:
        for i in range(10):
            sleep(0.5)
            self.senal_actualizar.emit(str(i))
        self.senal_actualizar.emit("Status: thread terminado")


class MiVentana(QWidget):
    # Creamos una señal para manejar la respuesta del thread
    senal_thread = pyqtSignal(str)

    def __init__(self) -> None:
        super().__init__()
        self.thread = None
        # Conectamos la señal al método que maneja
        self.senal_thread.connect(self.actualizar_label)
        self.init_gui()

    def init_gui(self) -> None:
        # Configuramos los widgets de la interfaz
        self.label = QLabel("Status: esperando thread", self)
        self.boton = QPushButton("Ejecutar Thread", self)
        self.boton.clicked.connect(self.ejecutar_thread)

        self.label.setGeometry(10, 10, 230, 30)
        self.boton.setGeometry(10, 50, 230, 30)

        # Configuramos las propiedades de la ventana.
        self.setWindowTitle("Ejemplo thread")
        self.setGeometry(50, 50, 250, 200)
        self.show()

    def ejecutar_thread(self) -> None:
        """
        Este método crea un thread cada vez que se presiona el botón en la
        interfaz. El thread recibirá como argumento la señal sobre la cual
        debe operar.
        """
        if self.thread is None or not self.thread.is_alive():
            self.thread = MiThread(self.senal_thread)
            self.thread.start()

    def actualizar_label(self, texto: str) -> None:
        """
        Este método actualiza el label según los datos enviados desde el
        thread a través del argumento texto. Para este ejemplo, el método
        recibe un texto, pero podría también no recibir nada.
        """
        self.label.setText(texto)


if __name__ == "__main__":

    def hook(type_, value, traceback):
        print(type_)
        print(traceback)

    sys.__excepthook__ = hook
    app = QApplication([])
    ventana = MiVentana()
    sys.exit(app.exec())
```

Con esto, y lo que sabemos del módulo `threading`, nos basta para generar comportamiento *multithread* en conjunto con nuestras interfaces, pero hay más. Debido a que PyQt se construye sobre la **arquitectura basada en manejo de eventos**, *threads* comunes y corrientes pueden generar problemas de concurrencia con esta arquitectura, por lo que PyQt provee su propia implementación de *threads*: el `QThread`.

## QThread

En términos simples, es un `Thread` como los del módulo `threading`. Provee prácticamente las mismas funcionalidades que ya conocemos, pero está creado dentro del ambiente de PyQt, por lo que es mucho más compatible con su arquitectura interna. Es parte del módulo `PyQt5.QtCore`, por lo que deberás importarlo desde ese módulo. A continuación se muestra una recreación del ejemplo anterior utilizando `QThread` en vez de `Thread`.

Este código se encuentra en el archivo `1-pyqt-qthreads/2_qthread.py`

```python
import sys
from time import sleep
from PyQt5.QtCore import pyqtSignal, QThread
from PyQt5.QtWidgets import QApplication, QWidget, QLabel, QPushButton


class MiThread(QThread):
    """
    Esta clase representa un thread personalizado que será utilizado durante
    la ejecución de la GUI.
    """

    def __init__(self, senal_actualizar: pyqtSignal) -> None:
        super().__init__()
        self.senal_actualizar = senal_actualizar

    def run(self) -> None:
        for i in range(10):
            sleep(0.5)
            self.senal_actualizar.emit(str(i))

        sleep(0.5)
        self.senal_actualizar.emit("Status: Qthread terminado")


class MiVentana(QWidget):
    # Creamos una señal para manejar la respuesta del thread
    senal_thread = pyqtSignal(str)

    def __init__(self) -> None:
        super().__init__()
        self.thread = None
        # Conectamos la señal del thread al método que maneja
        self.senal_thread.connect(self.actualizar_label)

        self.init_gui()

    def init_gui(self) -> None:
        # Configuramos los widgets de la interfaz
        self.label = QLabel("Status: esperando Qthread", self)
        self.boton = QPushButton("Ejecutar QThread", self)
        self.boton.clicked.connect(self.ejecutar_thread)

        self.label.setGeometry(10, 10, 230, 30)
        self.boton.setGeometry(10, 50, 230, 30)

        # Configuramos las propiedades de la ventana.
        self.setWindowTitle("Ejemplo Qthread")
        self.setGeometry(50, 50, 250, 200)
        self.show()

    def ejecutar_thread(self) -> None:
        """
        Este método crea un thread cada vez que se presiona el botón en la
        interfaz. El thread recibirá como argumento la señal sobre la cual
        debe operar.
        """
        # Aquí debemos ocupar isRunning en lugar de is_alive
        if self.thread is None or not self.thread.isRunning():
            self.thread = MiThread(self.senal_thread)
            self.thread.start()

    def actualizar_label(self, texto: str) -> None:
        """
        Este método actualiza el label según los datos enviados desde el
        thread a través del argumento texto. Para este ejemplo, el método
        recibe un texto, pero podría también no recibir nada.
        """
        self.label.setText(texto)


if __name__ == "__main__":

    def hook(type_, value, traceback):
        print(type_)
        print(traceback)

    sys.__excepthook__ = hook
    app = QApplication([])
    ventana = MiVentana()
    sys.exit(app.exec())
```

Se puede apreciar que se define de la misma forma que un `Thread`: se llama a `__init__` de la superclase de la cual se hereda, se define el método `run` que define el comportamiento al ser ejecutado, y este se ejecuta al llamar el método `start`. Una diferencia leve que se puede notar es que el método `is_alive` no existe en `QThread`, pero existe un equivalente llamado `isRunning`. En general, se puede encontrar un equivalente para todo método de `Thread` en `QThread`, y se pueden encontrar buscando documentación de la clase.

### QThread con su propia señal

Una ventaja que tiene el uso de `QThreads` es que ellos pueden tener sus propias señales definidas como atributos de clases. Esto **no es posible con Thread** dado que la creación de señales es exclusivo para objetos que sean parte del ecosistema PyQt. En el siguiente código se replicará el mismo ejemplo anterior, pero ahora la señal estará definida dentro de `MiThread(Qthread)` en vez de `MiVentana(QWidget)`. La única diferencia es que, una vez creado un `MiThread`, hay que conectar su señal con el método específico. Este código se encuentra en el archivo `1-pyqt-qthreads/3_qthread_con_signal.py`

```python
import sys
from time import sleep
from PyQt5.QtCore import pyqtSignal, QThread
from PyQt5.QtWidgets import QApplication, QWidget, QLabel, QPushButton


class MiThread(QThread):
    """
    Esta clase representa un thread personalizado que será utilizado durante
    la ejecución de la GUI.
    """

    # Creamos una señal para manejar la respuesta del thread
    senal_thread = pyqtSignal(str)

    def __init__(self) -> None:
        super().__init__()

    def run(self) -> None:
        for i in range(10):
            sleep(0.5)
            self.senal_thread.emit(str(i))

        sleep(0.5)
        self.senal_thread.emit("Status: Qthread terminado")


class MiVentana(QWidget):

    def __init__(self) -> None:
        super().__init__()
        self.thread = None
        self.init_gui()

    def init_gui(self) -> None:
        # Configuramos los widgets de la interfaz
        self.label = QLabel("Status: esperando Qthread", self)
        self.boton = QPushButton("Ejecutar QThread", self)
        self.boton.clicked.connect(self.ejecutar_thread)

        self.label.setGeometry(10, 10, 230, 30)
        self.boton.setGeometry(10, 50, 230, 30)

        # Configuramos las propiedades de la ventana.
        self.setWindowTitle("Ejemplo Qthread")
        self.setGeometry(50, 50, 250, 200)
        self.show()

    def ejecutar_thread(self) -> None:
        """
        Este método crea un thread cada vez que se presiona el botón en la
        interfaz. El thread recibirá como argumento la señal sobre la cual
        debe operar.
        """
        # Aquí debemos ocupar isRunning en lugar de is_alive
        if self.thread is None or not self.thread.isRunning():
            self.thread = MiThread()
            # Conectamos la señal del thread al método que maneja
            self.thread.senal_thread.connect(self.actualizar_label)
            self.thread.start()

    def actualizar_label(self, texto: str) -> None:
        """
        Este método actualiza el label según los datos enviados desde el
        thread a través del argumento texto. Para este ejemplo, el método
        recibe un texto, pero podría también no recibir nada.
        """
        self.label.setText(texto)


if __name__ == "__main__":

    def hook(type_, value, traceback):
        print(type_)
        print(traceback)

    sys.__excepthook__ = hook
    app = QApplication([])
    ventana = MiVentana()
    sys.exit(app.exec())
```

### Precaución con "donde" se conecta una señal

En el siguiente ejemplo, vamos a hacer que el código principal defina y ejecute un `Thread` (T1) y que dicho `Thread` defina y ejecute otro `Thread` (T2). Por lo tanto, tendremos 3 hilos: el principal, el de T1 y T2. Luego, vamos a conectar una señal en el hilo principal y otra en el hilo de T1. Finalmente, vamos a hacer que tanto T1 y T2 ejecuten las 2 señales. ¿Se ejecutarán todas como esperamos?

```python
from PyQt5.QtCore import QThread, pyqtSignal
from PyQt5.QtWidgets import QApplication
import sys
import time


class Thread1(QThread):
    signal_t1 = pyqtSignal(str)

    def __init__(self):
        super().__init__()
        self.t2 = None

    def run(self):
        print("[Thread 1] Crear Thread 2")
        self.t2 = Thread2(self.signal_t1)

        print("[Thread 1] Conectar Thread 2 (signar_t2) con print de T2")
        self.t2.signal_t2.connect(self.t2.print)

        print("[Thread 1] Ejecutar Thread 2")
        self.t2.start()

        print("[Thread 1] Emitir 2 señales: t1.signar_1 y t2.signal_2")
        self.signal_t1.emit("signal_t1 emitida desde Thread1")
        self.t2.signal_t2.emit("signal_t2 emitida desde Thread1")

        # Espero medio segundo y cierro el programa
        time.sleep(0.5)
        sys.exit(1)

    def print(self, s):
        print("\t[Print de T1] -> ", s)


class Thread2(QThread):
    signal_t2 = pyqtSignal(str)

    def __init__(self, signal_padre: pyqtSignal) -> None:
        super().__init__()
        self.signal_t1 = signal_padre

    def run(self):
        print("[Thread 2] Emitir 2 señales: t1.signar_1 y t2.signal_2")
        self.signal_t1.emit("signal_t1 emitida desde Thread2")
        self.signal_t2.emit("signal_t2 emitida desde Thread2")

    def print(self, s):
        print("\t[Print de T2] -> ", s)


if __name__ == "__main__":
    app = QApplication([])
    print("[MAIN] Crear Thread 1")
    hilo = Thread1()
    print("[MAIN] Conectar Thread 1 (signar_t1) con print de T1")
    hilo.signal_t1.connect(hilo.print)
    print("[MAIN] Ejecutar T1")
    hilo.start()
    sys.exit(app.exec())
```

Si ejecutamos este código, el resultado será:

```cmd
[MAIN] Crear Thread 1
[MAIN] Conectar Thread 1 (signar_t1) con print de T1
[MAIN] Ejecutar T1
[Thread 1] Crear Thread 2
[Thread 1] Conectar Thread 2 (signar_t2) con print de T2
[Thread 1] Ejecutar Thread 2
[Thread 1] Emitir 2 señales: t1.signar_1 y t2.signal_2
[Thread 2] Emitir 2 señales: t1.signar_1 y t2.signal_2
   [Print de T2] ->  signal_t2 emitida desde Thread1
   [Print de T1] ->  signal_t1 emitida desde Thread1
   [Print de T1] ->  signal_t1 emitida desde Thread2
```

En este ejemplo, se intentó emitir 4 señales, solo se ejecutaron 3. ¿Por qué?

Analizando con detalle, en el hilo principal (Main) se conectó `signal_t1` con el `print de T1`. Por lo tanto, cualquiera que emita la señal t1, logrará ejecutar el `print de T1`.

Respecto a la conexión de `signal_t2`, esta **no** fue conectada en el hilo principal, sino dentro del hilo de T1, en otras palabras, dentro del "mundo" o "dominio" de T1, esa conexión existe, pero fuera de dicho dominio, no se conoce dicha conexión. Por este motivo:
* Cuando T1 emite la señal `signal_t2`, se ejecuta el `print de T2` ya que justamente esa señal sí está conectada en el dominio de T1. 
* Cuando T2 emite la misma señal (`signal_t2`), en el dominio de T2 no existe esa conexión y tampoco es una conexión realizada en el hilo principal (Main). Por este motivo, no se ejecutó el `print de t2` cuando T2 emitió la señal `signal_t2`.

Por todo lo anterior, **se recomienda que toda señal sea conectada en el hilo principal (Main) y no lo haga un _thread_ secundario.**


### Múltiples *QThread*

Los ejemplos anteriores tenían 1 solo definido, pero ahora queremos tener una cantidad variable de `QThread` ejecutándose. Para esto, lo que debemos asegurarnos es **siempre** guardar una referencia, en memoria, al `QThread` y por lo mismo vamos a definir una lista donde se guardará cada `QThread` creado Este código se encuentra en el archivo `1-pyqt-qthreads/5_multiples_qthreads.py`


```python
import sys
from time import sleep
from PyQt5.QtCore import pyqtSignal, QThread
from PyQt5.QtWidgets import QApplication, QWidget, QLabel, QPushButton


class MiThread(QThread):
    # Se define para la clase MiThread,
    # para que cada instancia tenga una propia
    senal_actualizar = pyqtSignal(int, str)

    def __init__(self, indice: int, tiempo: float) -> None:
        super().__init__()
        self.indice = indice
        self.tiempo = tiempo

    def run(self) -> None:
        for i in range(10):
            sleep(self.tiempo)
            self.senal_actualizar.emit(self.indice, str(i))

        sleep(self.tiempo)
        self.senal_actualizar.emit(self.indice, "Status: QThread terminado")


class MiVentana(QWidget):
    def __init__(self) -> None:
        super().__init__()
        self.threads = []
        self.init_gui()

    def init_gui(self) -> None:
        # Configuramos los widgets de la interfaz
        # Definimos un montón de labels que corresponderán a un thread cada uno
        self.labels = {
            i: QLabel("Status: esperando QThread", self) for i in range(1, 6)
        }
        self.boton = QPushButton("Ejecutar QThreads", self)
        self.boton.clicked.connect(self.ejecutar_threads)

        for i in range(1, 6):
            self.labels[i].setGeometry(10, (i - 1) * 30, 330, 30)

        self.boton.setGeometry(10, 150, 330, 30)
        # Configuramos las propiedades de la ventana.
        self.setWindowTitle("Ejemplo Multiples QThreads")
        self.setGeometry(50, 50, 350, 200)
        self.show()

    def ejecutar_threads(self) -> None:
        """
        Este método crea cinco threads cada vez que se presiona el botón en la
        interfaz. Los threads recibirán como argumento el índice del label
        que les corresponde y el tiempo que toman entre cada iteración.
        """
        for thread in self.threads:
            if thread.isRunning():
                return

        self.threads = []
        for i in range(1, 6):
            thread = MiThread(i, i / 10)
            # Se conecta la señal emitida por el thread a un método
            # de la ventana
            thread.senal_actualizar.connect(self.actualizar_labels)
            self.threads.append(thread)
            thread.start()

    def actualizar_labels(self, indice: int, texto: str) -> None:
        """
        Este método actualiza el label correspondiente según los datos
        enviados desde un thread a través del índice y aplica el texto.
        """
        self.labels[indice].setText(texto)


if __name__ == "__main__":

    def hook(type_, value, traceback):
        print(type_)
        print(traceback)

    sys.__excepthook__ = hook
    app = QApplication([])
    ventana = MiVentana()
    sys.exit(app.exec())
```

### `QMutex`

Tal como vimos anteriormente, PyQt ofrece `QThread` como una versión propia del `Thread`. Esto mismo ocurre con los _locks_. Este objeto que nos permite controlar el acceso de múltiples _threads_ a una zona crítica de código también tiene su versión en PyQt. Este objeto se llama `QMutex` y mediante los métodos `lock()` y `unlock()` podemos garantizar que cierto fragmento de código solo sea ejecutado por 1 _thread_ a la vez.

A continuación mostraremos un ejemplo donde usaremos 6 _threads_ que actualizarán la ventana en diferentes tiempos. No obstante, el primer `for` que actualiza la ventana estará dentro de un `QMutex`, por lo que solo 1 _thread_ podrá ejecutar dicho código a la vez, mientras que un segundo `for`. que actualiza la ventana, no tendrá. `QMutex`. Esto hará que mientras uno o más threads estén ejecutando su segunda actualización, solo 1 podrá realizar la primera.

Este código se encuentra en el archivo `1-pyqt-qthreads/6_qmutex.py`.

```python
import sys
from time import sleep
from PyQt5.QtCore import pyqtSignal, QThread, QMutex
from PyQt5.QtWidgets import QApplication, QWidget, QLabel, QPushButton


class MiThread(QThread):
    # Se define para la clase MiThread,
    # para que cada instancia tenga una propia
    senal_actualizar = pyqtSignal(int, str)

    def __init__(self, indice: int, tiempo: float, mutex: QMutex) -> None:
        super().__init__()
        self.indice = indice
        self.tiempo = tiempo
        self.mutex = mutex

    def run(self) -> None:
        self.mutex.lock()
        for i in range(10):
            sleep(self.tiempo)
            self.senal_actualizar.emit(self.indice, str(i) + " - crítico")
        self.mutex.unlock()

        for i in range(50):
            sleep(self.tiempo)
            self.senal_actualizar.emit(self.indice, str(i) + " - no crítico")

        sleep(self.tiempo)
        self.senal_actualizar.emit(self.indice, "Status: QThread terminado")


class MiVentana(QWidget):
    def __init__(self) -> None:
        super().__init__()
        self.threads = []
        self.mutex = QMutex()
        self.init_gui()

    def init_gui(self) -> None:
        # Configuramos los widgets de la interfaz
        # Definimos un montón de labels que corresponderán a un thread cada uno
        self.labels = {
            i: QLabel("Status: esperando Qthread", self) for i in range(1, 6)
        }
        self.boton = QPushButton("Ejecutar QThreads", self)
        self.boton.clicked.connect(self.ejecutar_threads)

        for i in range(1, 6):
            self.labels[i].setGeometry(10, (i - 1) * 30, 230, 30)

        self.boton.setGeometry(10, 150, 230, 30)
        # Configuramos las propiedades de la ventana.
        self.setWindowTitle("Ejemplo QMutex")
        self.setGeometry(50, 50, 250, 200)
        self.show()

    def ejecutar_threads(self) -> None:
        """
        Este método crea cinco threads cada vez que se presiona el botón en la
        interfaz. Los threads recibirán como argumento el índice del label
        que les corresponde y el tiempo que toman entre cada iteración.
        """
        for thread in self.threads:
            if thread.isRunning():
                return

        self.threads = []
        for i in range(1, 6):
            thread = MiThread(i, i / 20, self.mutex)
            # Se conecta la señal emitida por el thread a un método
            # de la ventana
            thread.senal_actualizar.connect(self.actualizar_labels)
            self.threads.append(thread)
            thread.start()

    def actualizar_labels(self, indice: int, texto: str) -> None:
        """
        Este método actualiza el label correspondiente según los datos
        enviados desde un thread a través del índice y aplica el texto.
        """
        self.labels[indice].setText(texto)


if __name__ == "__main__":

    def hook(type_, value, traceback):
        print(type_)
        print(traceback)

    sys.__excepthook__ = hook
    app = QApplication([])
    ventana = MiVentana()
    sys.exit(app.exec())
```

## Ejemplo Aplicado

Ahora, vamos a aplicar conceptos de modularización, _back-end_ y _front-end_, `QThreads` para generar el siguiente ejemplo:

![](img/ejemplo_aplicado.gif)


En este ejemplo, tendremos diferentes cuadrados que van de un extremo a otro a diferente velocidad. Además, se puede hacer click en un cuadrado para cambiar su color y detener su movimiento. (Mini desafío: intenta detener todos los cuadrados).

Para esto, dentro del directorio `scripts` hay un directorio llamado `ejemplo_aplicado`. En este tenemos los siguientes archivos:

* `main.py`: es el archivo principal a ejecutar.
* `parametros_general.py`: son constantes que se ocuparán tanto en _back-end_ y _front-end_.

**backend**
* `backend/logica_qthread.py`: es el archivo principal del _back-end_ donde se aplica toda la lógica del movimiento de los cuadrados. En este archivo se ocupan `QThread` para modelar cada cuadrado.
* `backend/logica_thread.py`: es el archivo principal del _back-end_ donde se aplica toda la lógica del movimiento de los cuadrados. En este archivo se ocupan `threading.Thread` para modelar cada cuadrado. La única diferencia con `logica_qthread` es la herencia y que se debe explicitar que cada cuadrado es un _daemon_. 
* `backend/bonus_logica_qtimer.py`: es el mismo archivo que `logica_qthread.py` pero se utilizan `QTimer` en vez de `QThread`. Se recomienda ignorar este archivo para este ejemplo. Se utilizará en el contenido _bonus_.
* `backend/parametros_backend.py`: son constantes que se ocuparán únicamente en _back-end_.

**frontend**
* `frontend/ventana.py`: es el archivo principal del _front-end_ donde se crea la ventana y todo elemento visual.
* `frontend/parametros_backend.py`: son constantes que se ocuparán únicamente en _front-end_.

Para la correcta ejecución de este archivo, debes abrir tu terminal, desplazarte con `cd` hasta el directorio `scripts/ejemplo_aplicado/` y luego escribir `python3 main.py`.